### Import required libraries

In [1]:
import numpy as np
#import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import scipy.sparse.linalg as sla
import numpy.linalg as la
import sys, os
import pickle
#import rbf as RBF
import warnings

import geopandas #for importing data from shapefiles/geodb

#import vista #for plotting


### Import submodules for FME

In [2]:
from FME.geological_points import IePoint, IPoint, GPoint, TPoint
from FME.geological_interpolator import GeologicalInterpolator
from FME.tet_mesh import TetMesh
from FME.dsi_interpolator import DSI
from FME.foldframe import FoldFrame
from FME.fold import FoldEvent, DSIFoldConstraint
#from FME.structural_frame import StructuralFrame
from FME.fault_segment import FaultSegment
from FME.region_manager import RegionManager
from FME.helper import *
from FME.svariogram import s_variogram#
#from FME import * #import all of the "public" functions from FME

### Define the model area and mesh using tetmesh

In [3]:
boundary_points = np.zeros((2,3))
boundary_points[0,0] = -20
boundary_points[0,1] = -20
boundary_points[0,2] = -20
boundary_points[1,0] = 20
boundary_points[1,1] = 20
boundary_points[1,2] = 20
mesh = TetMesh('Model','./')
mesh.setup_mesh(boundary_points,nstep=1,n_tetra=100000)


### Create a DSI interpolation object for stratigraphy
* DSI needs to be given a within the mesh - by default this is everywhere which is the whole mesh.
* It needs to know the shape of the interpolation matrix e.g. do you want to assemble the least squares constraints as a rectangular matrix and solve 
$Ax=b A^TAx=A^TB$ 
or do you want to assemble $A^TA$ directly?
For standard DSI where only observations and a regularisation term are used either are fine but where multiple mesh wide constraints are used it can be beneficial to use the square format.
* There are multiple options for solvers that are directly incorporated into FME - for example scipy lsmr, lsqr, SuiteSpare SPQR, SuiteSparse Cholmod, scipy LU. In general a direct solver e.g. cholmod is the fastest option, however for really large meshes or small computers an iterative solver would be better. 
* There are some numerical problems using cholmod that need to be further investigated
* The weighting of the regularisation term should be defined relative to the number of elements in the mesh to avoid scaling issues when changing the size of the mesh. 

In [4]:
stratigraphy = DSI(mesh,region='everywhere',shape='square')
# for i in range(-15,15):
#     for j in range(-10,10):
#         stratigraphy.add_point([i,j,3*np.sin(j/5)],0.)
#         stratigraphy.add_point([i,j,3*np.sin(j/5)+1],1.)
# stratigraphy.add_strike_dip_and_value([2.1,0.1,1.1],190.,70.,0.)
for y in range(-18,18,1):
    stratigraphy.add_strike_dip_and_value([-10,y,-10],90.,0.,0.)
    stratigraphy.add_strike_dip_and_value([0,y,-6],90.,40.,0.)
    stratigraphy.add_strike_dip_and_value([5,y,7.1],90.,0.,0.)
#     stratigraphy.add_point([-10,y,-10],0.)
#     stratigraphy.add_point([0,y,-6],0.)
#     stratigraphy.add_point([5,y,7.1],0.)
stratigraphy.add_point([0.1,0,2.1],1.)
# stratigraphy.add_strike_and_dip([6.1,0.1,-15.1],90.,0.)
# stratigraphy.add_strike_and_dip([1.1,0.1,-15.1],90.,0.)
# stratigraphy.add_strike_and_dip([1.1,1.1,-15.1],90.,0.)
cgw = 100 / mesh.n_elements
stratigraphy.setup_interpolator(cgw=0.3)
# stratigraphy.calculate_constant_gradient_with_element_weighting(w)
stratigraphy.solve_system(solver='chol',clear=True)
mesh.update_property('strati',stratigraphy.c)
stratigraphy.export_data_to_vtk('stratig')

Setting up interpolator with 109 value control points 
        108 gradient control points and 0 tangent control points and 
        constant gradient regularization with a weight of 0.300000


### We can visualise isosurfaces of the interpolated scalar field using interactive lavavu
* lv_plot_isosurface takes a minimum of propertyname and isovalue. It will create a temporary lavavu viewer and plot a static image of the isosurface
* You can pass a lv object to the function using the argument lv and set draw to False and the surface will be added to the lavavu object. 
* The lavavu control panel can be used for interactively displaying object and an inline interactive viewer can be invoked by calling control.show()  

In [5]:
import lavavu
lv = lavavu.Viewer()
mesh.lv_plot_isosurface('strati',lv=lv,colour='red')
mesh.lv_plot_isosurface('strati',lv=lv,colour='green')
# mesh.lv_plot_isosurface('strati',2,name='Strati2',interactive=False,lv=lv,draw=False)

cp_points = lv.points("points",pointsize=4,pointtype="sphere")
cp_points.vertices(stratigraphy.get_control_points()[:,:3])#mesh.nodes[mesh.elements[logic]])
cp_points.values(stratigraphy.get_control_points()[:,3:])#mesh.nodes[mesh.elements[logic]])

gcp_points= lv.vectors("points2",pointsize=4,pointtype="sphere",colour='red')
gcp_points.vertices(stratigraphy.get_gradient_control()[:,:3])
gcp_points.vectors(stratigraphy.get_gradient_control()[:,3:])
lv.control.Panel()
lv.control.ObjectList()
    # lv.reload()
lv.control.show()#nterac
#lv.interactive()

NameError: name 'interactive' is not defined

### Static visualisation
* We can use the previous lavavu viewer to define a viewing angle and then display a static image for export

In [ ]:
#static display
lv.display()

lv.image('./data/interpolate_example.jpg')

In [ ]:
lv.interactive()

In [ ]:
grad_x, grad_y, grad_z = np.gradient(volume)

a = actual_verts[:, 0, :] - actual_verts[:, 1, :]
b = actual_verts[:, 0, :] - actual_verts[:, 2, :]

# Find triangle centroids
centroids = (actual_verts.sum(axis=1) / 3.).T

del actual_verts

# Interpolate face centroids into each gradient axis
grad_centroids_x = ndi.map_coordinates(grad_x, centroids)
grad_centroids_y = ndi.map_coordinates(grad_y, centroids)
grad_centroids_z = ndi.map_coordinates(grad_z, centroids)

# Combine and normalize interpolated gradients
grad_centroids = np.c_[grad_centroids_x, grad_centroids_y,
                       grad_centroids_z]
grad_centroids = (grad_centroids /
                  (np.sum(grad_centroids ** 2,
                          axis=1) ** 0.5)[:, np.newaxis])

# Find normal vectors for each face via cross product
crosses = np.cross(a, b)
crosses = crosses / (np.sum(crosses ** 2, axis=1) ** (0.5))[:, np.newaxis]

# Take dot product
dotproducts = (grad_centroids * crosses).sum(axis=1)

# Find mis-oriented faces
if 'descent' in gradient_direction:
    # Faces with incorrect orientations have dot product < 0
    indices = (dotproducts < 0).nonzero()[0]
elif 'ascent' in gradient_direction:
    # Faces with incorrection orientation have dot product > 0
    indices = (dotproducts > 0).nonzero()[0]
else:
    raise ValueError("Incorrect input %s in `gradient_direction`, see "
                     "docstring." % (gradient_direction))

# Correct orientation and return, without modifying original data
faces_corrected = faces.copy()
faces_corrected[indices] = faces_corrected[indices, ::-1]

return faces_corrected

In [ ]:
import lavavu  #visualisation library 
from FME.tet_mesh import marching_tetra
##run the marching tetra algorithm       
propertyname='strati'
isovalue=0
reg = np.zeros(mesh.properties[propertyname].shape).astype(bool)
reg[:] = True

tri, ntri = marching_tetra(isovalue,mesh.elements,mesh.nodes,reg,mesh.properties[propertyname])

##convert from memoryview to np array
tri = np.array(tri)
ntri = np.array(ntri)[0]
##create a triangle indices array and initialise to -1
tris = np.zeros((ntri,3)).astype(int)
tris[:,:] = -1
##create a dict for storing nodes index where the key is the node as as a tuple. 
#A dict is preferable because it is very quick to check if a key exists
#assemble arrays for unique vertex and triangles defined by vertex indices
nodes = {}
n = 0 #counter
for i in range(ntri):
    for j in range(3):
        if tuple(tri[i,j,:]) in nodes:
            tris[i,j] = nodes[tuple(tri[i,j,:])]
        else:
            nodes[tuple(tri[i,j,:])] = n
            tris[i,j] = n
            n+=1
nodes_np = np.zeros((n,3))
for v in nodes.keys():
    nodes_np[nodes[v],:] = np.array(v)
#if lv==None:
#    lv = lavavu.Viewer(border=True,quality=2)
surf = lv.triangles(propertyname)
surf.vertices(nodes_np)
surf.indices(tris)
# surf.colours(colour)

gcp_points= lv.vectors("points3",pointsize=4,pointtype="sphere",colour='red')
gcp_points.vertices(tribc)
gcp_points.vectors(crosses)

lv.control.Panel()
lv.control.ObjectList()
    # lv.reload()
lv.control.show()#nterac
#lv.interactive()

In [ ]:
#find the normal vector to the faces using the vertex order
a = tri[:ntri, 0, :] - tri[:ntri, 1, :]
b = tri[:ntri, 0, :] - tri[:ntri, 2, :]

crosses = np.cross(a, b)
crosses = crosses / (np.sum(crosses ** 2, axis=1) ** (0.5))[:, np.newaxis]

#get barycentre of faces and findproperty gradient from scalar field
tribc = np.mean(tri[:ntri,:,:],axis=1)

propertygrad=mesh.eval_gradient(tribc,prop='strati')
propertygrad/=np.linalg.norm(propertygrad,axis=1)[:,None]

#dot product between gradient and normal indicates if faces are incorrectly ordered
dotproducts = (propertygrad * crosses).sum(axis=1)

#if dot product >0 then adjust triangle indexing
indices = (dotproducts>0).nonzero()[0]
tris[indices] = tris[indices,::-1]



In [ ]:
print(tribc)
print(propertygrad)
print(dotproducts)
print(indices)
#tris[tris]

In [ ]:
lv = lavavu.Viewer()
# gcp_points= lv.vectors("points5",pointsize=4,pointtype="sphere",colour='red')
# gcp_points.vertices(tribc)
# gcp_points.vectors(crosses)

# gcp_points= lv.vectors("points6",pointsize=4,pointtype="sphere",colour='blue')
# gcp_points.vertices(tribc)
# gcp_points.vectors(propertygrad)

surf = lv.triangles(propertyname)
surf.vertices(nodes_np)
surf.indices(tris)
surf.colours('green')
lv.control.Panel()
lv.control.show()